###**Project - Omdena Retrolux Challenge**

#### Description of Code:
1) To populate the annual hours for projects with zero and null annual hours using the daily hours and weeks per year.

2) To add address, city, state, zipcodes, longitude and latitude to Features dataset.

### Import necessary packages and datasets

In [1]:
import pandas as pd 
# Mount Google Drive to Collaboratory 
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Read dataset into dataframes
df_features = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Final_complete.csv')
df_address = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Addresses_Geodata_Projects Merge - Sheet1.csv')

In [3]:
df_features.shape

(5425, 49)

#### Replace populate zero and null annual hours with daily hours and weeks per year

In [4]:
# Check number of rows with null annual hours 
df_features[df_features['ave_annual_hours'].isnull()].shape


(641, 49)

There are 641 rows with null annual hours 

In [5]:
# Check number of rows with zero annual hours 
df_features[df_features['ave_annual_hours']==0].shape

(1, 49)

Just one row has zero annual hours.


In [6]:
# Create a new column called average weekly hours as the sum of all the average daily hours
df_features ['ave_weekly_hours']=df_features ['monday_ave_daily_hours']+df_features ['tuesday_ave_daily_hours']+df_features ['wednesday_ave_daily_hours']+df_features ['thursday_ave_daily_hours']+df_features ['friday_ave_daily_hours']+df_features ['saturday_ave_daily_hours']+df_features ['sunday_ave_daily_hours']
df_features.head()

,Unnamed: 0,project_id,option_id,option_calculation_id,company_id,projects_name,facility_type_id,facility_type,industry_type_id,name,started_at,finished_at,tax_rate_source,finance_discount_rate,tax_exempt,active,area_count,area_count_sum,ave_annual_hours,monday_ave_daily_hours,tuesday_ave_daily_hours,wednesday_ave_daily_hours,thursday_ave_daily_hours,friday_ave_daily_hours,saturday_ave_daily_hours,sunday_ave_daily_hours,ave_weeks_per_year,hour_type,max_kwh_cost,max_kwh_cost_simple,existing_lighting_count,total_watts_per_product,ave_lm70,total_solution_count,total_existing_quantity,total_proposed_quantity,total_replacement_count,total_fixture_watts,total_fixture_watt_hours,total_fixture_life_years,total_item_watts,sol_total_existing_count,total_product_count,energy_savings_per_year,total_savings_per_year,rebates,net_replacement_cost,payback,ROI,ave_weekly_hours
0,0,3,8.0,1235.0,5,Eagle Enterprises,NaN,NaN,2252.0,"Trantow, Langworth and Moore",2020-03-22 05:34:45.016,2020-03-22 05:34:47.182041,NaN,5,False,True,2,42.0,4880.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,2,3276.0,13333.333333,2,56.0,56.0,56.0,0.0,0.0,0.000000,0.0,4,4,0.000000e+00,0.000000e+00,18.0,2.1300,NaN,0.000000,0.0
1,1,8,290.0,1398.0,7,Acme Warehouse,NaN,NaN,NaN,Bartoletti and Sons,2020-03-22 01:28:04.846642,2020-03-22 01:28:08.084044,NaN,5,False,True,1,1.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.1,NaN,1,100.0,1000.000000,1,10.0,10.0,10.0,13.0,130000.0,50.000000,130.0,1,1,8.700000e+01,3.536667e+02,NaN,119.0000,0.34,297.198880,0.0
2,2,11,92.0,1221.0,15,Golden Spoon_257,19.0,Fast Food Restaurants,NaN,"Anderson, Shields and Hyatt",2020-03-22 05:43:31.817303,2020-03-22 05:43:33.92774,0.0,5,False,True,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1080.0,10000.000000,1,86.0,86.0,43.0,600.0,0.0,0.000000,12900.0,2,2,0.000000e+00,0.000000e+00,NaN,13330.0000,NaN,0.000000,NaN
3,3,15,21.0,1782.0,19,Demo Project - updated and synced name - again,NaN,NaN,2336.0,"Blanda, Stiedemann and Kiehn",2021-04-28 21:40:38.078686,2021-04-28 21:40:43.529132,NaN,5,False,True,5,936.0,3727.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.1,NaN,5,662760.0,22538.461538,5,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,6,6,6.217249e-15,6.217249e-15,NaN,0.0000,NaN,inf,0.0
4,4,15,928.0,1781.0,19,Demo Project - updated and synced name - again,NaN,NaN,2336.0,"Blanda, Stiedemann and Kiehn",2020-03-22 01:55:33.805258,2020-03-22 01:55:40.416535,NaN,5,False,True,6,936.0,3727.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.1,NaN,6,795312.0,22538.461538,6,138.0,118.0,80.0,83.0,2097888.0,180.288462,1681.0,12,10,9.665942e+02,1.044594e+03,NaN,8489.2094,NaN,12.304965,0.0


In [7]:
# Replace the null annual hours with the product of weekly hours and weeks_per_year
df_features.loc[(df_features['ave_weekly_hours']>=0)& (df_features['ave_annual_hours'].isnull()),'ave_annual_hours']=df_features['ave_weekly_hours']*df_features['ave_weeks_per_year']
df_features[df_features['ave_annual_hours'].isnull()].shape

(268, 50)

The number of rows with null annual hours has reduced from 641 to 268.

In [8]:
df_features[df_features['ave_annual_hours'].isnull()]['ROI'].value_counts()

0.000000      245
0.361474        1
429.653740      1
Name: ROI, dtype: int64

The 268 rows also have null values in their weekly hours and weeks per year. Most of them also have zero ROI

In [9]:
# Replace the zero annual hours with the product of weekly hours and weeks_per_year
df_features.loc[(df_features['ave_weekly_hours']>=0)& (df_features['ave_annual_hours']==0),'ave_annual_hours']=df_features['ave_weekly_hours']*df_features['ave_weeks_per_year']
df_features[df_features['ave_annual_hours']==0].shape

(0, 50)

The only row with zero annual hours has been populated with the product of weekly hours and weeks per year.

In [10]:
df_features.head()

,Unnamed: 0,project_id,option_id,option_calculation_id,company_id,projects_name,facility_type_id,facility_type,industry_type_id,name,started_at,finished_at,tax_rate_source,finance_discount_rate,tax_exempt,active,area_count,area_count_sum,ave_annual_hours,monday_ave_daily_hours,tuesday_ave_daily_hours,wednesday_ave_daily_hours,thursday_ave_daily_hours,friday_ave_daily_hours,saturday_ave_daily_hours,sunday_ave_daily_hours,ave_weeks_per_year,hour_type,max_kwh_cost,max_kwh_cost_simple,existing_lighting_count,total_watts_per_product,ave_lm70,total_solution_count,total_existing_quantity,total_proposed_quantity,total_replacement_count,total_fixture_watts,total_fixture_watt_hours,total_fixture_life_years,total_item_watts,sol_total_existing_count,total_product_count,energy_savings_per_year,total_savings_per_year,rebates,net_replacement_cost,payback,ROI,ave_weekly_hours
0,0,3,8.0,1235.0,5,Eagle Enterprises,NaN,NaN,2252.0,"Trantow, Langworth and Moore",2020-03-22 05:34:45.016,2020-03-22 05:34:47.182041,NaN,5,False,True,2,42.0,4880.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,2,3276.0,13333.333333,2,56.0,56.0,56.0,0.0,0.0,0.000000,0.0,4,4,0.000000e+00,0.000000e+00,18.0,2.1300,NaN,0.000000,0.0
1,1,8,290.0,1398.0,7,Acme Warehouse,NaN,NaN,NaN,Bartoletti and Sons,2020-03-22 01:28:04.846642,2020-03-22 01:28:08.084044,NaN,5,False,True,1,1.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.1,NaN,1,100.0,1000.000000,1,10.0,10.0,10.0,13.0,130000.0,50.000000,130.0,1,1,8.700000e+01,3.536667e+02,NaN,119.0000,0.34,297.198880,0.0
2,2,11,92.0,1221.0,15,Golden Spoon_257,19.0,Fast Food Restaurants,NaN,"Anderson, Shields and Hyatt",2020-03-22 05:43:31.817303,2020-03-22 05:43:33.92774,0.0,5,False,True,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1080.0,10000.000000,1,86.0,86.0,43.0,600.0,0.0,0.000000,12900.0,2,2,0.000000e+00,0.000000e+00,NaN,13330.0000,NaN,0.000000,NaN
3,3,15,21.0,1782.0,19,Demo Project - updated and synced name - again,NaN,NaN,2336.0,"Blanda, Stiedemann and Kiehn",2021-04-28 21:40:38.078686,2021-04-28 21:40:43.529132,NaN,5,False,True,5,936.0,3727.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.1,NaN,5,662760.0,22538.461538,5,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,6,6,6.217249e-15,6.217249e-15,NaN,0.0000,NaN,inf,0.0
4,4,15,928.0,1781.0,19,Demo Project - updated and synced name - again,NaN,NaN,2336.0,"Blanda, Stiedemann and Kiehn",2020-03-22 01:55:33.805258,2020-03-22 01:55:40.416535,NaN,5,False,True,6,936.0,3727.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.1,NaN,6,795312.0,22538.461538,6,138.0,118.0,80.0,83.0,2097888.0,180.288462,1681.0,12,10,9.665942e+02,1.044594e+03,NaN,8489.2094,NaN,12.304965,0.0


In [11]:
df_features.columns

Index(['Unnamed: 0', 'project_id', 'option_id', 'option_calculation_id',
       'company_id', 'projects_name', 'facility_type_id', 'facility_type',
       'industry_type_id', 'name', 'started_at', 'finished_at',
       'tax_rate_source', 'finance_discount_rate', 'tax_exempt', 'active',
       'area_count', 'area_count_sum', 'ave_annual_hours',
       'monday_ave_daily_hours', 'tuesday_ave_daily_hours',
       'wednesday_ave_daily_hours', 'thursday_ave_daily_hours',
       'friday_ave_daily_hours', 'saturday_ave_daily_hours',
       'sunday_ave_daily_hours', 'ave_weeks_per_year', 'hour_type',
       'max_kwh_cost', 'max_kwh_cost_simple', 'existing_lighting_count',
       'total_watts_per_product', 'ave_lm70', 'total_solution_count',
       'total_existing_quantity', 'total_proposed_quantity',
       'total_replacement_count', 'total_fixture_watts',
       'total_fixture_watt_hours', 'total_fixture_life_years',
       'total_item_watts', 'sol_total_existing_count', 'total_product_count',

In [12]:
# drop the daily average hours, the weekly hours and weeks_per_year columns.
df_features.drop(columns = ['monday_ave_daily_hours', 'tuesday_ave_daily_hours','wednesday_ave_daily_hours', 'thursday_ave_daily_hours','friday_ave_daily_hours', 'saturday_ave_daily_hours',
       'sunday_ave_daily_hours', 'ave_weeks_per_year', 'hour_type','ave_weekly_hours'], inplace = True)
df_features.shape

(5425, 40)

#### Add Columns for Address, City, State, and Zipcode

In [13]:
df_address.head()

,created_by_user_id,active,addressable_id,addressable_type,address_type,address,city,state,zip_code,created_at,updated_at,geodata.source_id,geodata.name,geodata.county,geodata.state,geodata.zip_codes,geodata.location_type,geodata.latitude,geodata.longitude,geodata.area_code,geodata.population,geodata.households,geodata.median_income,geodata.land_area,geodata.water_area,geodata.time_zone,Projects.id,Projects.company_id,Projects.name,Projects.created_by_user_id,Projects.created_at,Projects.updated_at,Projects.company_project_status_id,Projects.client_id,Projects.contact_id,Projects.active,Projects.utility_id,Projects.utility_rate_plan,Projects.facility_type_id,Projects.rate_schedule_id,Projects.operating_schedule_id,Projects.cooling_id,Projects.heating_id,Projects.prepared_for_user_id,Projects.prepared_by_user_id,Projects.maintenance_labor_rate,Projects.markup,Projects.apply_tax_on,Projects.tax_type,Projects.tax_rate_source,Projects.tax_rate,Projects.encentiv_id,Projects.financing_debt_percentage,Projects.financing_interest_rate,Projects.incentive_max_type,Projects.incentive_max,Projects.all_company_access,Projects.client_logo,Projects.markup_type,Projects.test,Projects.probability,Projects.expected_close_date,Projects.shipping_address_location,Projects.rebate_bus_id,Projects.highest_value_option_id,Projects.audit_status,Projects.funding_status,Projects.enable_sync,Projects.finance_discount_rate,Projects.finance_term,Projects.copied_from_id,Projects.copy_status,Projects.tax_exempt,Projects.custom_attributes,Projects.measure_types
0,40,True,8433,Project,1,111 S Franklin Ave,Valley Stream,NY,11580,5/26/20 12:54,5/26/20 14:48,29433.0,North Valley Stream,Nassau County,New York,11580.0,CDP,40.68510,-73.70180,516.0,17247.0,5121.0,100882.0,4829049.0,84194.0,America/New York,8433,485,Orange Diner_16,1401,54:12.7,27:54.4,5374,NaN,2836.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False,NaN,NaN,1.0,NaN,NaN,NaN,NaN,False,5.0,10.0,6396.0,0.0,False,{},{lighting}
1,40,True,6784,Project,1,1150 W Lacey Blvd,Hanford,CA,93230,1/31/20 15:29,1/31/20 15:31,2470.0,Grangeville,Kings County,California,93230.0,CDP,36.34356,-119.70874,559.0,607.0,239.0,50450.0,1657015.0,0.0,America/Los Angeles,6784,485,Hanford West High School Cafeteria,1424,29:06.3,29:04.3,5374,NaN,2737.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,False,NaN,NaN,1.0,NaN,NaN,NaN,NaN,False,5.0,10.0,6396.0,0.0,False,{},{lighting}
2,40,True,6784,Project,1,1150 W Lacey Blvd,Hanford,CA,93230,1/31/20 15:29,1/31/20 15:31,2504.0,Hardwick,Kings County,California,93230.0,CDP,36.40134,-119.71874,559.0,184.0,47.0,18250.0,359363.0,0.0,America/Los Angeles,6784,485,Hanford West High School Cafeteria,1424,29:06.3,29:04.3,5374,NaN,2737.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,False,NaN,NaN,1.0,NaN,NaN,NaN,NaN,False,5.0,10.0,6396.0,0.0,False,{},{lighting}
3,40,True,6784,Project,1,1150 W Lacey Blvd,Hanford,CA,93230,1/31/20 15:29,1/31/20 15:31,2533.0,Home Garden,Kings County,California,93230.0,CDP,36.30328,-119.63624,559.0,1787.0,464.0,32976.0,1151090.0,0.0,America/Los Angeles,6784,485,Hanford West High School Cafeteria,1424,29:06.3,29:04.3,5374,NaN,2737.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,False,NaN,NaN,1.0,NaN,NaN,NaN,NaN,False,5.0,10.0,6396.0,0.0,False,{},{lighting}
4,1300,True,6277,Project,1,1900 Hazel St,Pine Bluff,AR,71603,12/6/19 19:15,1/3/20 20:47,1415.0,Sulphur Springs,Jefferson County,Arkansas,71603.0,CDP,34.18065,-92.12348,870.0,1260.0,559.0,41637.0,14450777.0,0.0,America/Chicago,6277,663,SEARK,1300,15:47.7,09:44.1,6425,NaN,NaN,True,3108.0,NaN,53.0,4536.0,8409.0,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,2.0,False,NaN,NaN,1.0,NaN,6248.0,NaN,NaN,True,5.0,10.0,NaN,0.0,False,{},{lighting}


In [14]:
# create a smaller dataframe with the relevant columns we need
df = df_address[['Projects.id','address','city','state','zip_code','geodata.latitude','geodata.longitude']]
df.rename(columns = {'Projects.id':'project_id','geodata.latitude':'latitude','geodata.longitude':'longitude'}, inplace = True ) # rename the project_id foreign to facilitate merge
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,project_id,address,city,state,zip_code,latitude,longitude
0,8433,111 S Franklin Ave,Valley Stream,NY,11580,40.68510,-73.70180
1,6784,1150 W Lacey Blvd,Hanford,CA,93230,36.34356,-119.70874
2,6784,1150 W Lacey Blvd,Hanford,CA,93230,36.40134,-119.71874
3,6784,1150 W Lacey Blvd,Hanford,CA,93230,36.30328,-119.63624
4,6277,1900 Hazel St,Pine Bluff,AR,71603,34.18065,-92.12348


In [15]:
# Merge the data set with our features data
df_features_final = pd.merge(df_features,df,on = 'project_id', how = 'left')
df_features_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5625 entries, 0 to 5624
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                5625 non-null   int64  
 1   project_id                5625 non-null   int64  
 2   option_id                 5625 non-null   float64
 3   option_calculation_id     5625 non-null   float64
 4   company_id                5625 non-null   int64  
 5   projects_name             5625 non-null   object 
 6   facility_type_id          2585 non-null   float64
 7   facility_type             2585 non-null   object 
 8   industry_type_id          2538 non-null   float64
 9   name                      5625 non-null   object 
 10  started_at                5625 non-null   object 
 11  finished_at               5625 non-null   object 
 12  tax_rate_source           5610 non-null   float64
 13  finance_discount_rate     5625 non-null   int64  
 14  tax_exem

Only 483 rows were populated with address, city, state and zip_code information and 356 rows with longitude and latitude information

In [16]:
df_features_final.shape

(5625, 46)

In [17]:
df_features_final.to_csv (r'/content/gdrive/MyDrive/Colab Notebooks/features_complete_final.csv', index = False, header=True)